In [1]:
from feature.clip_make_dataset import make_data
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

c:\Python\Python38\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [3]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [4]:
train_1 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.25.csv')
test_1 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.25.csv')
train_3 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.75.csv')
test_3 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.75.csv')
train_std = pd.read_csv('./data/clip_preprocessing/clip_train_std.csv')
test_std = pd.read_csv('./data/clip_preprocessing/clip_test_std.csv')
train_mid = pd.read_csv('./data/clip_preprocessing/clip_train_median.csv')
test_mid = pd.read_csv('./data/clip_preprocessing/clip_test_median.csv')

In [5]:
train_3.columns = [col+' 3' for col in train_3.columns]
test_3.columns = [col+' 3' for col in test_3.columns]
train_1.columns = [col+' 1' for col in train_1.columns]
test_1.columns = [col+' 1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
train_mid.columns = [col+' mid' for col in train_mid.columns]
test_mid.columns = [col+' mid' for col in test_mid.columns]

In [6]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
train_mid = train_mid.reset_index()
test_mid = test_mid.reset_index()


In [7]:
train = pd.concat([train, train_1, train_3, train_std, train_mid], axis=1)
test = pd.concat([test, test_1, test_3, test_std, test_mid], axis=1)

train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)

In [9]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [30]:
Case = train['Case']
X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
y = train['predicted_weight_g']

In [44]:
from sklearn.model_selection import GroupKFold

group = train['Case'].unique()

gkf = GroupKFold(n_splits=len(group))
gkf.get_n_splits(X, y, Case)
folds = []
for train_idx, val_idx in gkf.split(X, y, Case):
    folds.append((train_idx, val_idx))
Cat_model = {}


for f in range(28):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU")

    cat.fit(x_train, y_train)

    y_pred = cat.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    print(f"{f + 1} Fold RMSE = {rmse}")
    Cat_model[f] = cat
    print(f'================================================================================\n\n')
    

    

====================================1============================================
Learning rate set to 0.03894
0:	learn: 40.5105031	total: 13.8ms	remaining: 13.8s
100:	learn: 7.5067471	total: 896ms	remaining: 7.97s
200:	learn: 5.5496115	total: 1.78s	remaining: 7.1s
300:	learn: 5.0108580	total: 2.69s	remaining: 6.25s
400:	learn: 4.6308640	total: 3.6s	remaining: 5.38s
500:	learn: 4.2484852	total: 4.5s	remaining: 4.49s
600:	learn: 3.9721696	total: 5.44s	remaining: 3.61s
700:	learn: 3.6868140	total: 6.34s	remaining: 2.7s
800:	learn: 3.5573750	total: 7.21s	remaining: 1.79s
900:	learn: 3.4582422	total: 8.1s	remaining: 890ms
999:	learn: 3.3781710	total: 9.01s	remaining: 0us
1 Fold RMSE = 4.409625060894721


====================================2============================================
Learning rate set to 0.03894
0:	learn: 40.5000061	total: 9.94ms	remaining: 9.93s
100:	learn: 7.1177672	total: 934ms	remaining: 8.31s
200:	learn: 5.3722093	total: 1.89s	remaining: 7.53s
300:	learn: 4.8623921	t

In [49]:
x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
submit = test[['DAT', 'predicted_weight_g']]
submit['DAT'] = submit['DAT']+1

for fold in range(28):
    submit['predicted_weight_g'] += Cat_model[fold].predict(x_test)/28

all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)